In [ ]:
import numpy as np
import pandas as pd
from fractions import Fraction

In [80]:
comp_common_player_info = pd.read_csv('common_player_info.csv')
comp_player = pd.read_csv('player.csv')

In [ ]:
# Extraccion de datos
tables = ["common_player_info", "draft_combine_stats", "draft_history", "game_info", "game", "inactive_players",
          "line_score", "other_stats", "play_by_play", "player", "team_details", "team"]

# Cargar cada archivo CSV en un DataFrame con el nombre del elemento en `tables`
for table_name in tables:
    globals()[table_name] = pd.read_csv(f"{table_name}.csv")


In [ ]:
# Crear el archivo de texto y escribir la información de las tablas
with open("info_tablas.txt", "w") as file:
    file.write("TABLES: (registros/columnas)\n")
    
    # Recorrer la lista `tables` para agregar la información de cada DataFrame
    for i, table_name in enumerate(tables, start=1):
        df = globals()[table_name]  # Obtener el DataFrame correspondiente
        shape_info = f'{df.shape[0]}/{df.shape[1]}'  # Formato de (registros/columnas)
        
        # Escribir la información de la tabla
        file.write(f'      "{i}") "{table_name}": {shape_info}\n')
        file.write("           COLUMNAS: (formato)\n")
        
        # Recorrer cada columna para obtener el primer valor no nulo o "Sin datos disponibles"
        first_non_null_values = []
        for column in df.columns:
            # Obtener el primer valor no nulo de la columna o "Sin datos disponibles"
            first_non_null = df[column].dropna().iloc[0] if not df[column].dropna().empty else "Sin datos disponibles"
            first_non_null_values.append(first_non_null)
        
        # Escribir las columnas y los primeros valores no nulos
        file.write(f'           {list(df.columns)}: {first_non_null_values}\n')

In [ ]:
# Crear un diccionario vacío
column_dict = {}

# Recorrer cada nombre en la lista "tables"
for table_name in tables:
    # Obtener el DataFrame usando globals()
    df = globals()[table_name]
    
    # Recorrer las columnas del DataFrame
    for column in df.columns:
        # Si la columna no está en el diccionario, inicializa una lista
        if column not in column_dict:
            column_dict[column] = []
        
        # Agregar el nombre del DataFrame a la lista
        column_dict[column].append(table_name)

# Nombre del archivo donde se guardará la información
output_file = 'columnas_y_dataframes.txt'

# Abrir el archivo en modo escritura
with open(output_file, 'w') as file:
    # Recorrer el diccionario
    for column, dataframes in column_dict.items():
        # Escribir el índice y sus valores en el archivo
        file.write(f"{column}: {', '.join(dataframes)}\n")

print(f"Los datos se han guardado en {output_file}.")

In [ ]:
# Filtrar por año
for table_name in tables:
    if 'season' in globals()[table_name].columns:
        globals()[table_name] = globals()[table_name][globals()[table_name]['season'] > 1999]
    elif 'game_date' in globals()[table_name].columns:
        globals()[table_name]['game_date'] = pd.to_datetime(globals()[table_name]['game_date'])
        globals()[table_name] = globals()[table_name][globals()[table_name]['game_date'].dt.year > 1999]

In [ ]:
# Renombrar columnas para evitar ambiguedades y para igualar nombres de columnas que tienen los mismos atributos
rename_columns = {
    'common_player_info': {
        'person_id': 'player_id',
        'display_first_last': 'player_name',
        'draft_round': 'round_pick'
    },
    'draft_history': {
        'person_id': 'player_id'
    },
    'team': {
        'id': 'team_id',
        'full_name': 'team_name'
    },
    'player': {
        'id': 'player_id',
        'full_name': 'player_name'
    },
    'inactive_players': {
        'jersey_num': 'jersey'
    },
    'line_score': {
        'game_date_est': 'game_date'
    },
    'team_details': {
        'abbreviation': 'team_abbreviation',
        'nickname': 'team_nickname',
        'city': 'team_city',
        'yearfounded': 'year_founded'
    },
    'team_details': {
        'abbreviation': 'team_abbreviation',
        'nickname': 'team_nickname',
        'city': 'team_city',
        'state': 'team_state'
    }
}

# Apply renaming for each DataFrame
for df_name, renames in rename_columns.items():
    globals()[df_name].rename(columns=renames, inplace=True)


In [ ]:
# Filtrar los datos del resto de los df por los df ya filtrados
# Create the list of unique game_id values from the game DataFrame
unique_game_ids = game['game_id'].unique().tolist()
unique_player_ids = draft_combine_stats['player_id'].unique().tolist()
unique_team_ids = draft_history['team_id'].unique().tolist()


# Loop through each DataFrame name in the tables list, filtering based on game_id if the column exists
for table_name in tables:
    df = globals()[table_name]
    if 'player_id' in df.columns:
        globals()[table_name] = df[df['player_id'].isin(unique_player_ids)]
        
for table_name in tables:
    df = globals()[table_name]
    if 'team_id' in df.columns:
        globals()[table_name] = df[df['team_id'].isin(unique_team_ids)]

for table_name in tables:
    df = globals()[table_name]
    if 'game_id' in df.columns:
        globals()[table_name] = df[df['game_id'].isin(unique_game_ids)]


# Filter play_by_play DataFrame
play_by_play = play_by_play[
    play_by_play['player1_id'].isin(unique_player_ids) |
    play_by_play['player2_id'].isin(unique_player_ids) |
    play_by_play['player3_id'].isin(unique_player_ids)
]


In [99]:
# Asegúrate de que 'player_id' en ambos DataFrames esté en el mismo formato, por ejemplo, tipo int
common_player_info['player_id'] = common_player_info['player_id'].astype(int)
draft_combine_stats['player_id'] = draft_combine_stats['player_id'].astype(int)

# Realizar el merge entre common_player_info y draft_combine_stats basado en player_id
merged_df = common_player_info.merge(
    draft_combine_stats[['player_id', 'height_wo_shoes']],
    on='player_id',
    how='inner'
)

# Reemplazar directamente los valores de 'height' con 'height_wo_shoes'
merged_df['height'] = merged_df['height_wo_shoes']

# Eliminar la columna 'height_wo_shoes' ya que solo se usaba para llenar valores en 'height'
merged_df = merged_df.drop(columns=['height_wo_shoes'])

# Actualizar common_player_info con los registros actualizados
common_player_info = merged_df

# Mostrar el DataFrame actualizado
print(common_player_info)



     player_id          player_name  birthdate             school  \
0       203919         Jordan Adams 1994-07-08               UCLA   
1       203500         Steven Adams 1993-07-20         Pittsburgh   
2      1628959         Rawle Alkins 1997-10-29            Arizona   
3      1628959         Rawle Alkins 1997-10-29            Arizona   
4      1628959         Rawle Alkins 1997-10-29            Arizona   
..         ...                  ...        ...                ...   
108    1628414  Sindarius Thornwell 1994-11-15     South Carolina   
109    1628401        Derrick White 1994-07-02           Colorado   
110     203912          C.J. Wilcox 1990-12-30         Washington   
111    1626210        Alan Williams 1993-01-28  Cal-Santa Barbara   
112    1629684       Grant Williams 1998-11-30          Tennessee   

         country              last_affiliation  height      weight season_exp  \
0            USA                      UCLA/USA   75.50  209.00 lbs          3   
1    New 

In [ ]:
# Eliminar Columnas que no se vayan a usar
# List of DataFrame names and columns to drop
columns_to_drop = {
    'first_name': ['common_player_info', 'draft_combine_stats', 'inactive_players', 'player'],
    'last_name': ['common_player_info', 'draft_combine_stats', 'inactive_players', 'player'],
    'display_fi_last': ['common_player_info'],
    'display_last_comma_first': ['common_player_info'],
    'player_slug': ['common_player_info'],
    'facebook': ['team_details'],
    'instagram': ['team_details'],
    'twitter': ['team_details'],
    'neutraldescription': ['play_by_play'],
    'height_w_shoes_ft_in': ['draft_combine_stats'],
    'height_wo_shoes_ft_in': ['draft_combine_stats'],
    'wingspan_ft_in': ['draft_combine_stats'],
    'standing_reach_ft_in': ['draft_combine_stats']
}

# Loop through each column and its associated DataFrames
for column, dfs in columns_to_drop.items():
    for df_name in dfs:
        df = globals()[df_name]  # Access the DataFrame by name
        if column in df.columns:  # Check if the column exists
            df.drop(columns=[column], inplace=True)  # Drop the column
            print(f"Column '{column}' removed from {df_name}")

Column 'height_w_shoes_ft_in' removed from draft_combine_stats
Column 'height_wo_shoes_ft_in' removed from draft_combine_stats
Column 'wingspan_ft_in' removed from draft_combine_stats
Column 'standing_reach_ft_in' removed from draft_combine_stats


In [110]:
# Preparar columnas para el formateo
columnas_string = ['player_id', 'player_name', 'school', 'country', 'last_affiliation', 'jersey', 'position', 
'team_id', 'team_name', 'team_abbreviation', 'team_code', 'team_city', 'playercode',
'position', 'draft_type', 'organization', 'organization_type', 'game_id', 'team_id_home',
'team_abbreviation_home', 'team_name_home', 'matchup_home', 'team_id_away',
'team_abbreviation_away', 'team_name_away', 'matchup_away', 'season_type', 'team_city_name_home',
'team_nickname_home', 'team_city_name_away', 'team_nickname_away', 'league_id', 'eventnum',
'eventmsgtype', 'eventmsgactiontype', 'homedescription', 'visitordescription', 'score',
'person1type', 'player1_id', 'player1_name', 'player1_team_id', 'player1_team_city', 
'player1_team_nickname', 'player1_team_abbreviation', 'player2_id', 'player2_name',
'player2_team_id', 'player2_team_city', 'player2_team_nickname', 'player2_team_abbreviation',
'player3_id', 'player3_name', 'player3_team_id', 'player3_team_city', 'player3_team_nickname',
'player3_team_abbreviation', 'owner', 'generalmanager', 'headcoach', 'dleagueaffiliation', 'team_state',
'season_exp', 'from_year', 'to_year', 'draft_year', 'round_pick', 'draft_number', 'season', 'round_number',
'overall_pick', 'season_id', 'arena']
columnas_bool = ['rosterstatus', 'games_played_current_season_flag', 'dleague_flag', 'nba_flag',
'games_played_flag', 'greatest_75_flag', 'player_profile_flag', 'wl_home', 'video_available_home',
'wl_away', 'video_available_away', 'video_available_flag', 'is_active']
columnas_date = ['birthdate', 'game_date']
columnas_timestring = ['game_time', 'wctimestring', 'pctimestring']
columnas_pulgadas = ['height', 'height_wo_shoes', 'height_w_shoes', 'wingspan', 'standing_reach', 'hand_length',
'hand_width', 'standing_vertical_leap', 'max_vertical_leap']
columnas_libras = ['weight']
columnas_float3 = ['body_fat_pct', 'fg_pct_home', 'fg3_pct_home', 'ft_pct_home', 'fg_pct_away', 'fg3_pct_away', 'ft_pct_away']
columnas_tiempo = ['lane_agility_time', 'modified_lane_agility_time', 'three_quarter_sprint']
columnas_int = ['bench_press', 'attendance', 'fgm_home',
'fga_home', 'fg3m_home', 'fg3a_home', 'ftm_home', 'fta_home', 'oreb_home', 'dreb_home',
'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pts_home', 'plus_minus_home',
'fgm_away', 'fga_away', 'fg3m_away', 'fg3a_away', 'ftm_away', 'fta_away', 'oreb_away',
'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away', 'pf_away', 'pts_away',
'plus_minus_away', 'game_sequence', 'pts_qtr1_home', 'pts_qtr2_home', 'pts_qtr3_home',
'pts_qtr4_home', 'pts_ot1_home', 'pts_ot2_home', 'pts_ot3_home', 'pts_ot4_home', 'pts_ot5_home',
'pts_ot6_home', 'pts_ot7_home', 'pts_ot8_home', 'pts_ot9_home', 'pts_ot10_home', 'pts_home',
'pts_qtr1_away', 'pts_qtr2_away', 'pts_qtr3_away', 'pts_qtr4_away', 'pts_ot1_away',
'pts_ot2_away', 'pts_ot3_away', 'pts_ot4_away', 'pts_ot5_away', 'pts_ot6_away', 'pts_ot7_away',
'pts_ot8_away', 'pts_ot9_away', 'pts_ot10_away', 'pts_away', 'pts_paint_home', 'pts_2nd_chance_home',
'pts_fb_home', 'largest_lead_home', 'lead_changes', 'times_tied', 'team_turnovers_home',
'total_turnovers_home', 'team_rebounds_home', 'pts_off_to_home', 'pts_paint_away',
'pts_2nd_chance_away', 'pts_fb_away', 'largest_lead_away', 'team_turnovers_away',
'total_turnovers_away', 'team_rebounds_away', 'pts_off_to_away', 'period', 'scoremargin',
'yearfounded', 'arenacapacity']
columnas_fracciones = ['spot_fifteen_corner_left', 'spot_fifteen_break_left', 'spot_fifteen_top_key',
'spot_fifteen_break_right', 'spot_fifteen_corner_right', 'spot_college_corner_left',
'spot_college_break_left', 'spot_college_top_key', 'spot_college_break_right',
'spot_college_corner_right', 'spot_nba_corner_left', 'spot_nba_break_left',
'spot_nba_top_key', 'spot_nba_break_right', 'spot_nba_corner_right',
'off_drib_fifteen_break_left', 'off_drib_fifteen_top_key', 'off_drib_fifteen_break_right',
'off_drib_college_break_left', 'off_drib_college_top_key', 'off_drib_college_break_right',
'on_move_fifteen', 'on_move_college']
columnas_ratios = ['team_wins_losses_home', 'team_wins_losses_away']


In [ ]:
# Transformar columnas de fracciones a su formato correcto
dataframes = [draft_combine_stats, line_score]  # your target dataframes

# Define the conversion function
def to_fraction(value):
    if isinstance(value, str) and '-' in value:
        parts = value.split('-')
        if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
            x, y = parts
            return Fraction(int(x), int(y))
    # Return original value if not in 'x-y' format or if malformed
    return value

# Apply the transformation to each specified column in each dataframe
for df in dataframes:
    for col in columnas_fracciones:
        if col in df.columns:
            df[col] = df[col].apply(to_fraction)

# Now 'x-y' formatted strings in specified columns are converted to Fraction objects

In [ ]:
# Filtrar draft_combine_stats para obtener solo los registros que tengan informacion acerca de las pruebas de tiro en alguna de las ligas
# Create a mask for rows with at least 5 non-null values in 'columnas_fracciones'
mask = draft_combine_stats[columnas_fracciones].notna().sum(axis=1) >= 5

# Apply the mask to filter out rows that don't meet the condition
draft_combine_stats = draft_combine_stats[mask].copy()

print("Filtered DataFrame shape:", draft_combine_stats.shape)


Filtered DataFrame shape: (233, 41)


In [ ]:
# Rellenar valores nulos de common_player_info, draft_combine_stats, game_info, inactive_players, line_score, other_stats
# Merge the DataFrames on player_id
common_player_info.fillna(0, inplace=True)
draft_combine_stats.fillna(0, inplace=True)
game_info.fillna(0, inplace=True)
inactive_players.fillna(0, inplace=True)
line_score.fillna(0, inplace=True)
other_stats.fillna(0, inplace=True)

In [ ]:
# Rellenar valores nulos en game (wl_home, ft_pct_home, wl_away, ft_pct_away)
# Aquí definimos la lógica para rellenar wl_home y wl_away
def set_wl(row):
    if row['pts_home'] > row['pts_away']:
        return pd.Series(['W', 'L'])  # wl_home, wl_away
    else:
        return pd.Series(['L', 'W'])  # wl_home, wl_away

# Aplicamos la función y rellenamos las columnas correspondientes
game[['wl_home', 'wl_away']] = game.apply(set_wl, axis=1)


def calculate_pct(row):
    row['fg_pct_home'] = row['fgm_home'] / row['fga_home'] if row['fga_home'] != 0 else 0
    row['fg_pct_away'] = row['fgm_away'] / row['fga_away'] if row['fga_away'] != 0 else 0
    row['fg3_pct_home'] = row['fg3m_home'] / row['fg3a_home'] if row['fg3a_home'] != 0 else 0
    row['fg3_pct_away'] = row['fg3m_away'] / row['fg3a_away'] if row['fg3a_away'] != 0 else 0
    row['ft_pct_home'] = row['ftm_home'] / row['fta_home'] if row['fta_home'] != 0 else 0
    row['ft_pct_away'] = row['ftm_away'] / row['fta_away'] if row['fta_away'] != 0 else 0
    return row

# Aplicamos la función a cada fila
game = game.apply(calculate_pct, axis=1)

# Verificamos el resultado
game.isnull().sum()


In [85]:
# Rellenar valores nulos en play_byplay

# Realizamos un merge de 'play_by_play' con 'draft_history' usando 'player3_id' y 'player_id'
merged_df = play_by_play.merge(
    comp_player[['id', 'full_name']],
    how='left',
    left_on='player1_id',
    right_on='id'
)

# Rellenamos los valores nulos de las columnas de 'play_by_play' con los valores correspondientes de 'draft_history'
merged_df['player1_name'] = merged_df['player1_name'].combine_first(merged_df['full_name'])

# Eliminamos las columnas adicionales para regresar al formato original
play_by_play = merged_df[play_by_play.columns]

# Realizamos un merge de 'play_by_play' con 'draft_history' usando 'player3_id' y 'player_id'
merged_df = play_by_play.merge(
    comp_common_player_info[['person_id', 'display_first_last', 'team_code', 'team_id', 'team_city', 'team_abbreviation']],
    how='left',
    left_on='player3_id',
    right_on='person_id'
)

# Rellenamos los valores nulos de las columnas de 'play_by_play' con los valores correspondientes de 'draft_history'
merged_df['player3_name'] = merged_df['player3_name'].combine_first(merged_df['display_first_last'])
merged_df['player3_team_nickname'] = merged_df['player3_team_nickname'].combine_first(merged_df['team_code'])
merged_df['player3_team_id'] = merged_df['player3_team_id'].combine_first(merged_df['team_id'])
merged_df['player3_team_city'] = merged_df['player3_team_city'].combine_first(merged_df['team_city'])
merged_df['player3_team_abbreviation'] = merged_df['player3_team_abbreviation'].combine_first(merged_df['team_abbreviation'])

# Eliminamos las columnas adicionales para regresar al formato original
play_by_play = merged_df[play_by_play.columns]

# Realizamos un merge de 'play_by_play' con 'draft_history' usando 'player2_id' y 'player_id'
merged_df = play_by_play.merge(
    comp_common_player_info[['person_id', 'display_first_last', 'team_code', 'team_id', 'team_city', 'team_abbreviation']],
    how='left',
    left_on='player2_id',
    right_on='person_id'
)

# Rellenamos los valores nulos de las columnas de 'play_by_play' con los valores correspondientes de 'draft_history'
merged_df['player2_name'] = merged_df['player2_name'].combine_first(merged_df['display_first_last'])
merged_df['player2_team_nickname'] = merged_df['player2_team_nickname'].combine_first(merged_df['team_code'])
merged_df['player2_team_id'] = merged_df['player2_team_id'].combine_first(merged_df['team_id'])
merged_df['player2_team_city'] = merged_df['player2_team_city'].combine_first(merged_df['team_city'])
merged_df['player2_team_abbreviation'] = merged_df['player2_team_abbreviation'].combine_first(merged_df['team_abbreviation'])

# Eliminamos las columnas adicionales para regresar al formato original
play_by_play = merged_df[play_by_play.columns]




In [ ]:
# Rellenar resto de valores nulos con N/A ya que son strings entonces permiten ser rellenados de esa forma
for table_name in tables:
    globals()[table_name].fillna('N/A', inplace=True)

In [108]:
# Recorremos cada DataFrame en la lista 'tables'
for df in tables:
    # Reemplazamos 'N/A' por 0 en las columnas que están en 'columnas_int'
    for column in columnas_int:
        if column in globals()[df].columns:
            globals()[df][column] = globals()[df][column].replace('N/A', 0)
            globals()[df][column] = globals()[df][column].replace('TIE', 0)

In [ ]:
# Aplicacion de formatos correspondientes
# Recorrer cada DataFrame en la lista tables
for df in tables:
    
    # Convertir columnas a string
    for col in globals()[df].columns.intersection(columnas_string + columnas_bool + columnas_timestring):
        globals()[df][col] = globals()[df][col].astype(str)
    
    # Convertir columnas a datetime
    for col in globals()[df].columns.intersection(columnas_date):
        globals()[df][col] = pd.to_datetime(globals()[df][col])  # 'coerce' convierte valores inválidos a NaT
    
    # Aplicar formato para columnas en pulgadas
    for col in globals()[df].columns.intersection(columnas_pulgadas):
        globals()[df][col] = globals()[df][col].astype(float).map(lambda x: f"{x:.2f} in")
    
    # Aplicar formato para columnas en libras
    for col in globals()[df].columns.intersection(columnas_libras):
        globals()[df][col] = globals()[df][col].astype(float).map(lambda x: f"{x:.2f} lbs")
    
    # Aplicar formato para columnas float con 3 decimales
    for col in globals()[df].columns.intersection(columnas_float3):
        globals()[df][col] = globals()[df][col].astype(float).round(3)
    
    '''# Aplicar formato para columnas de tiempo en segundos
    for col in globals()[df].columns.intersection(columnas_tiempo):
        globals()[df][col] = globals()[df][col].astype(float).map(lambda x: f"{x:.3f} segs")'''
    
    # Convertir columnas a enteros
    for col in globals()[df].columns.intersection(columnas_int):
        globals()[df][col] = globals()[df][col].astype(int)

In [112]:
# Define your conversion functions
def convert_inches(value):
    if isinstance(value, str):
        return round(float(value.replace('in', '').strip()), 2)
    return round(value, 2) if value is not None else None

def convert_pounds(value):
    if isinstance(value, str):
        return round(float(value.replace('lbs', '').strip()), 2)
    return round(value, 2) if value is not None else None

def convert_time(value):
    if isinstance(value, str):
        return round(float(value.replace('segs', '').strip()), 3)
    return round(value, 3) if value is not None else None


# Apply the conversions to each DataFrame
for df in tables:
    for col in columnas_pulgadas:
        if col in globals()[df].columns:
            globals()[df][col] = globals()[df][col].apply(convert_inches)

    for col in columnas_libras:
        if col in globals()[df].columns:
            globals()[df][col] = globals()[df][col].apply(convert_pounds)

    for col in columnas_tiempo:
        if col in globals()[df].columns:
            globals()[df][col] = globals()[df][col].apply(convert_time)

In [ ]:
# Create a single ExcelWriter instance
file_path = 'NBA_Filtrado2.xlsx'

with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    # Write each DataFrame to a separate sheet
    common_player_info.to_excel(writer, index=False, sheet_name='common_player_info')
    print('1')

# Now using a new 'with' block but still writing to the same file
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    draft_combine_stats.to_excel(writer, index=False, sheet_name='draft_combine_stats')
    print('2')

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    draft_history.to_excel(writer, index=False, sheet_name='draft_history')
    print('3')

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    game_info.to_excel(writer, index=False, sheet_name='game_info')
    print('4')

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    game.to_excel(writer, index=False, sheet_name='game')
    print('5')

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    inactive_players.to_excel(writer, index=False, sheet_name='inactive_players')
    print('6')

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    line_score.to_excel(writer, index=False, sheet_name='line_score')
    print('7')

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    other_stats.to_excel(writer, index=False, sheet_name='other_stats')
    print('8')

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    play_by_play.to_excel(writer, index=False, sheet_name='play_by_play')
    print('9')

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    player.to_excel(writer, index=False, sheet_name='player')
    print('10')

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    team_details.to_excel(writer, index=False, sheet_name='team_details')
    print('11')

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    team.to_excel(writer, index=False, sheet_name='team')
    print('12')

In [113]:
for table_name in tables:
    print(f'{table_name}: {globals()[table_name].shape}')



common_player_info: (113, 28)
draft_combine_stats: (233, 41)
draft_history: (112, 14)
game_info: (27457, 4)
game: (30426, 55)
inactive_players: (11072, 7)
line_score: (27457, 43)
other_stats: (25165, 26)
play_by_play: (726115, 33)
player: (179, 3)
team_details: (25, 11)
team: (30, 7)
